In [1]:
import numpy as np
import scanorama
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

scanpy==1.4 anndata==0.6.18 numpy==1.16.2 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [2]:
import utils_viz

In [3]:
# Create folder to save the results 
import os
dirname = os.getcwd()
print(dirname)
data_dir = os.path.join(dirname, './data/dataset1_uc3/')
if not os.path.exists('./results/'): os.makedirs('./results/')
save_dir = os.path.join(dirname, './results/')
def save_images(basename):
    if not os.path.exists('./figures/'): os.makedirs('./figures/')
    outname = './figures/' + basename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

/data_n3/home/hoatran/python-workspace/dca_scGen


In [4]:
# read data from read count text table, data from R: genes x cells
# adata = sc.read_text(os.path.join(data_dir,'dataset1_sm_uc3.txt'),delimiter='\t',first_column_names=True,dtype='float64')
# print(adata)
# print(len(adata.obs_names))
# print(adata.var_names)
# Read sample into a pandas series
# sample_adata = pd.read_csv(os.path.join(data_dir,'sample_sm_uc3.txt'),header=0, index_col=0, sep='\t')
# print(sample_adata.values.shape)
# print(sample_adata.keys())

# Data have format genes x cells
# But input for scanpy package require the format of cells x genes, samples x features
# We need to copyt data into new adata object with inverse raw data 
# adata1 = sc.AnnData(adata.X.T)
# adata1.var_names = adata.obs_names
# adata1.obs_names = adata.var_names
# adata1
# Save label information into adata object, same idea as Seurat 
# adata1.obs['cell_type'] = sample_adata.loc[adata1.obs_names,['celltype']]
# adata1.obs['batch'] = sample_adata.loc[adata1.obs_names,['batch']]
# print(len(adata1.obs['cell_type']))
# print(len(adata1.obs['batch']))
# Save output into h5ad, easy to access 
# adata1.write_h5ad(os.path.join(data_dir,'dataset1_uc3.h5ad'))


# sc.pp.filter_cells(adata1, min_genes=300)
# sc.pp.filter_genes(adata1, min_cells=5)
# adata1.write_h5ad(os.path.join(data_dir,'filtered_dataset1_uc3.h5ad'))
# print('Read and filter data')
# adata = sc.read_h5ad(os.path.join(data_dir,'filtered_dataset1_uc3.h5ad'))
# adata

In [5]:
print('Read and filter data')
adata = sc.read_h5ad(os.path.join(data_dir,'dataset1_uc3.h5ad'))
import time
from datetime import timedelta
t1 = time.time()
# Filtering data 
sc.pp.filter_cells(adata, min_genes=300)
sc.pp.filter_genes(adata, min_cells=5)
adata

Read and filter data
filtered out 11 cells that have less than 300 genes expressed
filtered out 9999 genes that are detected in less than 5 cells


AnnData object with n_obs × n_vars = 565 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'

In [6]:
from scanorama import correct, visualize, process_data
from scanorama import dimensionality_reduce

In [7]:
# df = pd.DataFrame(data=adata.X,
#                   index=adata.obs_names,
#                   columns=adata.var_names)

# Input of scanorama is a list of AnnData
adata1 = adata[adata.obs['batch']=='Batch1',:].copy()
print(adata1)
adata2 = adata[adata.obs['batch']=='Batch2',:].copy()
print(adata2)
# df1 = pd.DataFrame(data=adata1.X,
#                   index=adata1.obs_names,
#                   columns=adata1.var_names)
# df2 = pd.DataFrame(data=adata2.X,
#                   index=adata2.obs_names,
#                   columns=adata2.var_names)

AnnData object with n_obs × n_vars = 282 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'
AnnData object with n_obs × n_vars = 283 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'


In [8]:
# adata_ls = [df1.values, df2.values]
adata_ls = [adata1, adata2]
# df2.values.__class__

In [9]:
# Batch correction.
# datasets = [df.values]
# List of datasets (matrices of cells-by-genes)
corrected = scanorama.correct_scanpy(adata_ls,batch_size=30,return_dense=True, knn=10)
# corrected, genes = scanorama.correct(adata_ls, adata1.var_names,batch_size=30)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))

Found 16594 genes among all datasets
[[0.         0.63829787]
 [0.         0.        ]]
Processing datasets (0, 1)
Took 0:00:04.803442


In [11]:
print(corrected[0])
print(corrected[0].obs_names[0:10]==adata1.obs_names[0:10])

AnnData object with n_obs × n_vars = 282 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'
[ True  True  True  True  True  True  True  True  True  True]


In [12]:
print(corrected[1])
print(corrected[1].obs_names[0:10]==adata2.obs_names[0:10])

AnnData object with n_obs × n_vars = 283 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'
[ True  True  True  True  True  True  True  True  True  True]


In [13]:
total_ann = sc.AnnData(np.concatenate([corrected[0].X, corrected[1].X]))
print(total_ann)
# 

AnnData object with n_obs × n_vars = 565 × 16594 


In [14]:
total_ann.var_names = corrected[0].var_names
total_ann.obs_names = corrected[0].obs_names.tolist() + corrected[1].obs_names.tolist()
total_ann.obs['cell_type'] = corrected[0].obs['cell_type'].tolist() + corrected[1].obs['cell_type'].tolist()
total_ann.obs['batch'] = corrected[0].obs['batch'].tolist() + corrected[1].obs['batch'].tolist()
total_ann


AnnData object with n_obs × n_vars = 565 × 16594 
    obs: 'cell_type', 'batch'

In [16]:
sc.tl.pca(total_ann, svd_solver='arpack', n_comps=20)
print('before: ',total_ann.obsm['X_pca'][1:10])
total_ann.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
print('after multiply: ',total_ann.obsm['X_pca'][1:10])


before:  [[-2.18141854e-01  4.13227588e-01 -9.12025347e-02  2.68561728e-02
   2.62002517e-02  6.15926972e-03  6.68267310e-02 -6.32517971e-03
   1.36426613e-02  1.08428616e-02  3.61582488e-02 -1.46457488e-02
  -1.75916310e-02  2.96080336e-02  5.49193919e-02 -2.67209914e-02
   2.21198462e-02  9.01064556e-03 -3.34797456e-04  2.86887996e-02]
 [-2.55762517e-01  3.60720545e-01 -1.10248730e-01  3.11519280e-02
  -1.47297829e-02  6.97683915e-02  3.05099250e-03 -3.09731141e-02
  -1.55355232e-02 -7.20540434e-03  1.90493129e-02  1.82783771e-02
  -1.01564769e-02 -5.44375787e-03  1.89111903e-02 -2.31207125e-02
   3.03548817e-02  1.19124791e-02 -1.95491053e-02  1.87960658e-02]
 [-2.41118222e-01  3.34462970e-01 -1.35888994e-01 -4.73562367e-02
  -1.33338245e-02  3.88271511e-02 -9.51345712e-02  1.36554450e-01
  -4.19270471e-02 -6.92622364e-02 -4.29488253e-03 -6.30397797e-02
  -9.46711935e-03 -6.90273494e-02  3.59646343e-02  2.87710745e-02
  -4.64090556e-02 -8.45582858e-02  1.13773897e-01  6.89934241e-03

In [17]:
if not os.path.exists('./results/scanorama_results/'): os.makedirs('./results/scanorama_results/')
save_dir = os.path.join(dirname, 'results/scanorama_results/')
total_ann.write_h5ad(os.path.join(save_dir,'scanorama_dataset1_uc3.h5ad'))
total_ann.write_csvs(save_dir, skip_data=False)

... storing 'cell_type' as categorical
... storing 'batch' as categorical
writing '.csv' files to /data_n3/home/hoatran/python-workspace/dca_scGen/results/scanorama_results


In [19]:
print(t1)
print(t2)

1556848934.93988
1556848939.743322


In [20]:
# Function to plot TSNE
def plotTSNE(adata, color_group, n_pcs=20, perplexity=90, save_filename='tsne', use_repx = False):
    #adata.var_names_make_unique()
    if use_repx:
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity, use_rep='X')
    else:    
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity)
    sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
    save_images(save_filename)     
    
    
def plotUMAP(adata, color_group, save_filename, use_repx = False):
    
    if use_repx:
        sc.pp.neighbors(adata, use_rep='X')
    else:    
        sc.pp.neighbors(adata,n_neighbors=10, n_pcs=20)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename) 
    
   

In [21]:
# Visulization of normalized data
# sc.tl.pca(total_ann, svd_solver='arpack')
sc.pp.neighbors(total_ann,n_neighbors=10, n_pcs=20)
print('scanorama - batch effect correction')
color_group = ["batch","cell_type"]
plotUMAP(total_ann, color_group, 'umap_scanorama_corrected_dataset1_uc3')
plotTSNE(total_ann, color_group, 20, 90, 'tsne_scanorama_corrected_dataset1_uc3')

computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:05.07) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
scanorama - batch effect correction
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:00.17) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
computing UMAP
    finished (0:00:02.89) --> added
    'X_umap', UMAP coordinates (adata.obsm)
computing tSNE
    using 'X_pca' with n_pcs = 20
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished (0:00:04.30) --> added
    'X_tsne', tSNE coordinates (adata.obsm)


In [ ]:
# scanorama_ann.obsm['X_umap'].__class__
colnu = []
for i in range(scanorama_ann.obsm['X_umap'].shape[1]):
    colnu.append("UMAP"+str(i+1))
df = pd.DataFrame(scanorama_ann.obsm['X_umap'], columns=colnu, index=scanorama_ann.obs_names)
df['batch'] = pd.Series(scanorama_ann.obs['batch'], index=scanorama_ann.obs_names)
df['celltype'] = pd.Series(scanorama_ann.obs['cell_type'], index=scanorama_ann.obs_names)
df.to_csv(sdir_name+'scanorama_umap.csv')  

In [ ]:
colnt = []
for i in range(scanorama_ann.obsm['X_tsne'].shape[1]):
    colnt.append("tSNE_"+str(i+1))

df = pd.DataFrame(scanorama_ann.obsm['X_tsne'], columns=colnt, index=scanorama_ann.obs_names)
df['batch'] = pd.Series(scanorama_ann.obs['batch'], index=scanorama_ann.obs_names)
df['celltype'] = pd.Series(scanorama_ann.obs['cell_type'], index=scanorama_ann.obs_names)
df.to_csv(sdir_name+'scanorama_tsne.csv')  

In [ ]:
sdir_name = os.path.join(dirname, 'results/scanorama_results/')
scanorama_ann = sc.read_h5ad(os.path.join(sdir_name,'scanorama_dataset1_uc3.h5ad'))
scanorama_ann.obsm['X_pca'].shape
print(scanorama_ann.obsm['X_pca'][1:5])
def write_to_csv(mat, genesname, cellsname, filename, save_dir):
    if isinstance(mat, np.ndarray):
        df = pd.DataFrame(mat, columns=genesname, index=cellsname)
    else:
        df = pd.DataFrame(mat.toarray(), columns=genesname, index=cellsname)        
    
    df.to_csv(save_dir+filename)  
    
filename = 'scanorama_pca.csv'
coln_pca = []
for i in range(scanorama_ann.obsm['X_pca'].shape[1]):
    coln_pca.append("X_pca"+str(i+1))
    

write_to_csv(scanorama_ann.obsm['X_pca'], coln_pca, scanorama_ann.obs_names,filename, data_dir)

In [1]:
# Compute the silhouette coefficient score, compare the batch mixing level of filtered data and normalized data
from sklearn.metrics import silhouette_score
import seaborn as sns
sc.tl.pca(adata, svd_solver='arpack', n_comps=20)
adata.obsm['X_pca'] *= -1 # multiply by -1 to match Seurat, same scale as total_ann
sil_orig = silhouette_score(adata.obsm.X_pca[:, :20], adata.obs['batch'])
print(np.median(sil_orig))
# sil_true = silhouette_score(corrected_adata.obsm.X_pca[:, :2], adata_true.obs.Group)
sil_corr = silhouette_score(total_ann.obsm.X_pca[:, :20], total_ann.obs['batch'])
print(np.median(sil_corr))
sns.barplot(x=['Filtered', 'Normalized'], y=[sil_orig, sil_corr])
pl.title('Silhouette Coeff Batch Scanorama')
save_images('scanorama_silhouette_coeff_batch_label') 

NameError: name 'sc' is not defined

In [103]:
# Check the changement of values before scGen and after batch correction using scGen 
import scipy.stats as st
def describe_data(adata, ax, indx):    
    desc = st.describe(adata.X[indx], axis=ax)
    print('# of observations:', desc.nobs)
    print('min: %d\nmax: %d' % desc.minmax)
    print('mean: %.1f' % desc.mean)
    
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 10 in ",adata.obs['batch'][10],adata.obs_names[10])
print("Gene expression values before scanorama:")
# describe_data(adata, 0, 10) #cell id 10 in batch 1
print(adata.X[10])
print("Cell id: 10 in ",total_ann.obs['batch'][10],total_ann.obs_names[10])
print("Gene expression values after scanorama:")
print(total_ann.X[10])
# describe_data(total_ann, 0, 10) #cell id 10 in batch 1

Cell id: 10 in  Batch1 pDC_P10_S83
Gene expression values before scanorama:
[ 4.34  0.    7.49 ...  0.   98.93  0.  ]
Cell id: 10 in  Batch1 pDC_P10_S83
Gene expression values after scanorama:
[ 1.02420045e-05  6.38164929e-05  3.20880557e-04 ... -7.40184696e-05
  1.81876542e-03  8.92117314e-05]


In [105]:
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 500 in ",adata.obs['batch'][500],adata.obs_names[500])
print("Gene expression values before scanorama:")
describe_data(adata, 0, 500) #cell id 10 in batch 1
print(adata.X[500])
print("Cell id: 500 in ",total_ann.obs['batch'][500],total_ann.obs_names[500])
print("Gene expression values after scanorama:")
print(total_ann.X[500])
describe_data(total_ann, 0, 500) #cell id 10 in batch 

Cell id: 500 in  Batch2 CD1C_P14_S78
Gene expression values before scanorama:
# of observations: 16594
min: 0
max: 135216
mean: 60.2
[0. 0. 0. ... 0. 0. 0.]
Cell id: 500 in  Batch2 CD1C_P14_S78
Gene expression values after scanorama:
[0. 0. 0. ... 0. 0. 0.]
# of observations: 16594
min: 0
max: 0
mean: 0.0


In [106]:
adata

AnnData object with n_obs × n_vars = 565 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'
    uns: 'batch_colors', 'cell_type_colors', 'neighbors'
    obsm: 'X_pca', 'X_umap'

In [113]:
adata.obsm.X_pca[:,:20].shape

(565, 20)

0.17667641
0.007139875


In [129]:
pl.figure()
pl.boxplot([ sil_orig, sil_corr], showmeans=True, whis='range')
pl.title('Distributions of Silhouette Coefficients')
pl.xticks(range(1, 4), [ 'Filtered data', 'Scanorama'])
pl.ylabel('Silhouette Coefficient')
save_images('test')